In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Data Science/알파코 딥러닝 부트캠프/프로젝트/GPT 시 생성기'

In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 36.0 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 85.4 MB/s 
     |████████████████████████████████| 6.6 MB 71.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 34.8 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
     |████████████████████████████████| 1.1 MB 63.2 MB/s 
     |████████████████████████████████| 140 kB 86.8 MB/s 
     |████████████████████████████████| 212 kB 81.6 MB/s 
     |████████████████████████████████| 127 kB 86.6 MB/s 
     |████████████████████████████████| 271 kB 97.6 MB/s 
     |█████████████

In [ ]:
!gdown 1AFy749AzBM6n5arCbQmOsTeP7-5m8IoD

Downloading...
From: https://drive.google.com/uc?id=1AFy749AzBM6n5arCbQmOsTeP7-5m8IoD
To: /content/시 데이터1-799페이지.csv
100% 29.1M/29.1M [00:00<00:00, 79.0MB/s]


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, DataCollatorWithPadding
from datasets import load_metric, Dataset
import torch
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/시 데이터1-799페이지.csv')

In [ ]:
# data = df.clean_content

In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
dataset = dataset.remove_columns(['Unnamed: 0', 'title', 'content', 'author'])

In [ ]:
# dataset['clean_content']

In [ ]:
#@title 기본 제목 텍스트
test_size = "" #@param {type:"string"}
variable_name = ""
dataset = dataset.train_test_split()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2",
    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
    pad_token='<pad>', mask_token='<mask>')

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
# model = AutoModelForCausalLM.from_pretrained(path + '/results/checkpoint-3991/')

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["clean_content"]], truncation=True, padding='max_length', max_length=256)

In [ ]:
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=dataset["train"].column_names,
)

#0:   0%|          | 0/4 [00:00<?, ?ba/s]

#1:   0%|          | 0/4 [00:00<?, ?ba/s]

#2:   0%|          | 0/4 [00:00<?, ?ba/s]

#3:   0%|          | 0/4 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 15964
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3991
    })
})

In [ ]:
block_size = 16

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)

#0:   0%|          | 0/4 [00:00<?, ?ba/s]

#1:   0%|          | 0/4 [00:00<?, ?ba/s]

#2:   0%|          | 0/4 [00:00<?, ?ba/s]

#3:   0%|          | 0/4 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length', max_length=16)

In [ ]:
training_args = TrainingArguments(
    output_dir=path+"/results",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_eval_batch_size=64,
    per_device_train_batch_size=64,
    num_train_epochs=3.0,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

In [ ]:
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rat

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 255424
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 11973


Epoch,Training Loss,Validation Loss
1,2.044200,2.001364
2,1.975000,1.981925


***** Running Evaluation *****
  Num examples = 63856
  Batch size = 64
Saving model checkpoint to /content/drive/MyDrive/Data Science/알파코 딥러닝 부트캠프/프로젝트/GPT 시 생성기/results/checkpoint-3991
Configuration saved in /content/drive/MyDrive/Data Science/알파코 딥러닝 부트캠프/프로젝트/GPT 시 생성기/results/checkpoint-3991/config.json
Model weights saved in /content/drive/MyDrive/Data Science/알파코 딥러닝 부트캠프/프로젝트/GPT 시 생성기/results/checkpoint-3991/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 63856
  Batch size = 64
Saving model checkpoint to /content/drive/MyDrive/Data Science/알파코 딥러닝 부트캠프/프로젝트/GPT 시 생성기/results/checkpoint-7982
Configuration saved in /content/drive/MyDrive/Data Science/알파코 딥러닝 부트캠프/프로젝트/GPT 시 생성기/results/checkpoint-7982/config.json
Model weights saved in /content/drive/MyDrive/Data Science/알파코 딥러닝 부트캠프/프로젝트/GPT 시 생성기/results/checkp

KeyboardInterrupt: ignored

In [ ]:
# tokenizer.save_pretrained(path + '/saved')
# model.save_pretrained(path + '/saved')

Configuration saved in /content/drive/MyDrive/Data Science/알파코 딥러닝 부트캠프/프로젝트/GPT 시 생성기/saved/config.json
Model weights saved in /content/drive/MyDrive/Data Science/알파코 딥러닝 부트캠프/프로젝트/GPT 시 생성기/saved/pytorch_model.bin


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(path + '/saved')

In [ ]:
model = AutoModelForCausalLM.from_pretrained(path + '/saved')

In [ ]:
text = '꽃'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                           max_length=64,
                           repetition_penalty=1.5,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=False)
generated = tokenizer.decode(gen_ids[0])
print(generated)

꽃   피 우 고 파 라 니 요   그 대 로 인 해 서 도    내 가    나 를    사 랑 하 는 데     너 에 게 만 이 야 말 없 어 냐 며      아 무 리   정
